In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import logging
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [3]:
driver = webdriver.Chrome(service=Service(executable_path="/opt/homebrew/bin/chromedriver"), options=options)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="109462571935a0c51f1dc11be3b1d0a1")>

In [8]:
table_rows = driver.find_elements(By.TAG_NAME, 'tr')
type(table_rows)
len(table_rows)
# table_rows


21

In [9]:
def wait_for_element(locator, locator_type, timeout):
    try:
        WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located((locator_type, locator)))
        return True
    except TimeoutException:
        return False

In [10]:
def wait_for_and_find_element(locator, locator_type, timeout):
    try:
        wait = wait_for_element(locator, locator_type, timeout)
        if not wait:
            logging.error(f'Tried to wait to locate element via locator "{locator}", but timed out')
            return False, None
        element = driver.find_element(locator_type, locator)
        if not element:
            logging.error(f'Could not find element via locator "{locator}"')
            return True, None
        if wait and element:
            logging.info(f'Found and located element via locator "{locator}"')
            return True, element

    except Exception as NoSuchElementException:
        logging.exception(f'An unexpected error occurred: {NoSuchElementException}')


In [11]:
def find_element_and_click(locator ,locator_type=By.CSS_SELECTOR):
    """
    Finds element and clicks it using `WebElement.click()`
    :param locator:
    :param locator_type:
    :return: Tuple(bool, WebElement)
    """
    try:
        element = driver.find_element(locator_type, locator)
        if not element:
            logging.error(f'Could not locate element via locator "{locator}".')
        element.click()
        return True, element
    except NoSuchElementException:
        logging.exception(f'Element {locator} was not found.')
        return False, None
    except Exception as e:
        logging.exception(f'Error occurred when trying to find and click element with locator: "{locator}" resulting in error message: {str(e)}')
        return False, None


In [12]:
def scrape_wrapper():
    results = []
    for idx, table_row in enumerate(table_rows):
        if idx >= 1:
            table_row.click()
            found, elem = wait_for_and_find_element(locator="//*[@class='styles__SidesheetContent-sc-czzuxh-2 hKVVOI']", locator_type=By.XPATH, timeout=10)
            
            results.append(elem.text)
            
            exit_btn_clicked = find_element_and_click(locator='//*[@id="MerchantApp"]/div/div/div[3]/div[2]/div[2]/div/div/div[1]/nav/div[1]/div[1]/div/button', locator_type=By.XPATH)
            
            if exit_btn_clicked:
                logging.info(f'Exiting sidesheetbody for Order #: {idx}')
            
    return results


In [13]:
orders_content = scrape_wrapper()

In [68]:
orders_content

['Order: DF8E5AB6\nDelivered\nThe order was delivered at 10:56 PM on September 24, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nNikolas W\nDasher\nTommy\nPicked Up\n10:35 PM\n(Quoted: 10:12 PM)\nSeptember 24, 2023\nDropped Off\n10:56 PM\n(Quoted: 10:35 PM)\nSeptember 24, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nTXB Water Bottle (16.9 oz) (Water TX)\n$1.99\n5\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nDr Pepper\n$0.00\nSubtotal\n$1.99\nSubtotal Tax\n$0.00\nCommission (16%)\n-$0.32\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$1.67\nAssociated Transactions (1)\nTransaction #8058359503 - Delivery\n$1.67',
 'Order: 1D4B1698\nDelivered\nThe order was delivered at 7:18 PM on September 24, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nLisa C\nDasher\nEric\nPicked Up\n7:11 PM\n(Quoted: 6:57 PM)\nSeptember 24, 2023\nDropped Off\n7:18 PM\n(Qu

In [15]:
orders_content

['Order: 7587448E\nDelivered\nThe order was delivered at 11:15 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nmary r\nDasher\nMaurice\nPicked Up\n11:00 PM\n(Quoted: 10:55 PM)\nSeptember 25, 2023\nDropped Off\n11:15 PM\n(Quoted: 11:22 PM)\nSeptember 25, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nTXB Water Bottle (1gal) (Water TX)\n$6.58\nSubtotal\n$6.58\nSubtotal Tax\n$0.00\nCommission (16%)\n-$1.05\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$5.53\nAssociated Transactions (1)\nTransaction #8065464878 - Delivery\n$5.53',
 "Order: 51371086\nDelivered\nThe order was delivered at 11:07 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nChris W\nDasher\nLawrence\nPicked Up\n10:57 PM\n(Quoted: 10:37 PM)\nSeptember 25, 2023\nDropped Off\n11:07 PM\n(Quoted: 11:00 PM)\nSeptember 25, 2023\nPick Up Location\n102 W Main St, Calera, O

In [19]:
orders_content = ['Order: 7587448E\nDelivered\nThe order was delivered at 11:15 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nmary r\nDasher\nMaurice\nPicked Up\n11:00 PM\n(Quoted: 10:55 PM)\nSeptember 25, 2023\nDropped Off\n11:15 PM\n(Quoted: 11:22 PM)\nSeptember 25, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nTXB Water Bottle (1gal) (Water TX)\n$6.58\nSubtotal\n$6.58\nSubtotal Tax\n$0.00\nCommission (16%)\n-$1.05\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$5.53\nAssociated Transactions (1)\nTransaction #8065464878 - Delivery\n$5.53',
 "Order: 51371086\nDelivered\nThe order was delivered at 11:07 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nChris W\nDasher\nLawrence\nPicked Up\n10:57 PM\n(Quoted: 10:37 PM)\nSeptember 25, 2023\nDropped Off\n11:07 PM\n(Quoted: 11:00 PM)\nSeptember 25, 2023\nPick Up Location\n102 W Main St, Calera, OK 74730, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nCoke Cherry Bottle (20oz) (Soda OK)\n$3.79\n1\n×\nBen & Jerry's Chubby Hubby Ice Cream (16 oz) (Ice Cream OK)\n$12.49\nSubtotal\n$16.28\nSubtotal Tax\n$1.47\nCommission (23%)\n-$3.74\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$12.54\nAssociated Transactions (1)\nTransaction #8065451843 - Delivery\n$12.54",
 "Order: 85BA5F93\nDelivered\nThe order was delivered at 10:42 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nCornelius S\nDasher\nLawrence\nPicked Up\n10:24 PM\n(Quoted: 10:03 PM)\nSeptember 25, 2023\nDropped Off\n10:42 PM\n(Quoted: 10:38 PM)\nSeptember 25, 2023\nPick Up Location\n102 W Main St, Calera, OK 74730, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nM&M's Peanut King Size (3.27 oz) (Chocolate OK)\n$3.79\n1\n×\nHershey's Almond King Size (2.6 oz) (Chocolate OK)\n$3.79\n2\n×\nDr Pepper Bottle (20 oz) (Soda OK)\n$7.58\n2\n×\nPepsi Bottle (20 oz) (Soda OK)\n$7.58\nSubtotal\n$22.74\nSubtotal Tax\n$2.05\nCommission (16%)\n-$3.64\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$19.10\nAssociated Transactions (1)\nTransaction #8065275398 - Delivery\n$19.10",
 'Order: 85BA5F93',
 'Order: 7C78538C\nCustomer Picked Up\nThe customer picked up the order at 10:02 PM on September 25, 2023. \nGet help\nDelivery Details\nCustomer\nKes M\nPicked Up\n10:02 PM\n(Quoted: 10:13 PM)\nSeptember 25, 2023\nPick Up Location\n2503 TX-349, Midland, TX 79706, USA\nChannel\nDoorDash\nFulfillment\nConsumer Pickup\nOrder Details\n2\n×\nCheetos Crunchy (3.25 oz) (Chips & Popcorn TX)\n$4.58\n1\n×\nTwix Ice Cream Bar (3 oz) (Ice Cream TX)\n$3.99\n1\n×\nTXB Green Tea Bottle (16oz) (Tea TX)\n$2.59\nSubtotal\n$11.16\nSubtotal Tax\n$0.82\nCommission (9%)\n-$1.00\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$10.98\nAssociated Transactions (1)\nTransaction #8065155860 - Pickup\n$10.98',
 'Order: 792BC6EF\nDelivered\nThe order was delivered at 9:50 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nAaron E\nDasher\nArmando\nPicked Up\n9:29 PM\n(Quoted: 9:21 PM)\nSeptember 25, 2023\nDropped Off\n9:50 PM\n(Quoted: 9:48 PM)\nSeptember 25, 2023\nPick Up Location\n2196 El Indio Hwy, Eagle Pass, TX 78852, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nMiller Lite Beer Can (12 oz x 18 ct) (Domestic Beer ST)\n$27.49\nSubtotal\n$27.49\nSubtotal Tax\n$3.50\nCommission (20%)\n-$5.50\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$25.49\nAssociated Transactions (1)\nTransaction #8064944545 - Delivery\n$25.49',
 'Order: B0DFD67A\nDelivered\nThe order was delivered at 9:30 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nHillary R\nDasher\nGloria\nPicked Up\n9:23 PM\n(Quoted: 9:18 PM)\nSeptember 25, 2023\nDropped Off\n9:30 PM\n(Quoted: 9:39 PM)\nSeptember 25, 2023\nPick Up Location\n2320 W Main St, Durant, OK 74701, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nAir Heads Bites (2oz) (Fruity OK)\n$4.99\n2\n×\nSkittles Original King Size (4 oz) (Fruity OK)\n$6.78\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nDr Pepper\n$0.00\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nTXB Cherry Limeade\n$0.00\n2\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nTXB Fresh Brewed Sweet Tea\n$0.00\n1\n×\nTwix Caramel King Size (3 oz) (Chocolate OK)\n$3.79\n2\n×\nDr Pepper Bottle (20 oz) (Soda OK)\n$7.58\n1\n×\nBarcel Takis Blue Heat (9.9oz) (Chips & Popcorn OK)\n$7.49\nSubtotal\n$30.63\nPromo Paid by DoorDash\n-$6.00\n(DashPass Welcome Offer )\nSubtotal for Tax Calculations\n$24.63\nSubtotal Tax\n$2.31\nCommission (23%)\n-$7.04\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$23.59\nAssociated Transactions (1)\nTransaction #8064911188 - Delivery\n$23.59',
 'Order: E308F16E\nDelivered\nThe order was delivered at 8:18 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nRachel D\nDasher\nPolo\nPicked Up\n8:05 PM\n(Quoted: 8:06 PM)\nSeptember 25, 2023\nDropped Off\n8:18 PM\n(Quoted: 8:22 PM)\nSeptember 25, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nDr Pepper\n$0.00\n1\n×\nIce Large (Fountain, Frozen, & Tea)\n$7.49\nSubtotal\n$7.49\nSubtotal Tax\n$0.95\nCommission (16%)\n-$1.20\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$7.24\nAssociated Transactions (1)\nTransaction #8064275507 - Delivery\n$7.24',
 'Order: 6BFA2F11\nDelivered\nThe order was delivered at 7:13 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nKristy G\nDasher\nJLS\nPicked Up\n7:03 PM\n(Quoted: 7:09 PM)\nSeptember 25, 2023\nDropped Off\n7:13 PM\n(Quoted: 7:27 PM)\nSeptember 25, 2023\nPick Up Location\n801 Lafayette St, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nWhole Milk Vitamin D (1 gal) (Milk TX)\n$7.49\nSubtotal\n$7.49\nSubtotal Tax\n$0.00\nCommission (16%)\n-$1.20\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$6.29\nAssociated Transactions (1)\nTransaction #8063675206 - Delivery\n$6.29',
 'Order: 6BFA2F11',
 'Order: A308E7B3\nDelivered\nThe order was delivered at 5:25 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nDollymae J\nDasher\nAndrew\nPicked Up\n5:06 PM\n(Quoted: 5:03 PM)\nSeptember 25, 2023\nDropped Off\n5:25 PM\n(Quoted: 6:06 PM)\nSeptember 25, 2023\nPick Up Location\n14620 Ranch Rd 12, Wimberley, TX 78676, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n3\n×\nCayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)\n$17.07\nSubtotal\n$17.07\nSubtotal Tax\n$2.18\nCommission (23%)\n-$3.93\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$15.32\nAssociated Transactions (1)\nTransaction #8062580206 - Delivery\n$15.32',
 'Order: 9A76A89B\nDelivered\nThe order was delivered at 3:44 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nDebi G\nDasher\nEric\nPicked Up\n3:39 PM\n(Quoted: 3:39 PM)\nSeptember 25, 2023\nDropped Off\n3:44 PM\n(Quoted: 3:55 PM)\nSeptember 25, 2023\nPick Up Location\n1741 TX-121, Bonham, TX 75418, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nMountain Dew\n$0.00\n2\n×\nSkirt Steak Fajita Quesadilla (Quesadillas TXB#10)\n$14.98\nSubtotal\n$14.98\nSubtotal Tax\n$1.91\nCommission (20%)\n-$3.00\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$13.89\nAssociated Transactions (1)\nTransaction #8061960485 - Delivery\n$13.89',
 'Order: 9A76A89B',
 'Order: 8908C4FE',
 'Order: 6ABEF86F\nDelivered\nThe order was delivered at 2:23 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nJessie V\nDasher\nDanay\nPicked Up\n2:07 PM\n(Quoted: 1:57 PM)\nSeptember 25, 2023\nDropped Off\n2:23 PM\n(Quoted: 2:29 PM)\nSeptember 25, 2023\nPick Up Location\n2503 TX-349, Midland, TX 79706, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nRed Bull Red Edition Watermelon Can (12 oz) (Energy Drinks TX)\n$11.58\n1\n×\nSnickers King Size (3.29oz) (Chocolate TX)\n$3.79\nSubtotal\n$15.37\nSubtotal Tax\n$1.92\nCommission (16%)\n-$2.46\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$14.83\nAssociated Transactions (1)\nTransaction #8061354366 - Delivery\n$14.83',
 'Order: 96F7D0D1\nDelivered\nThe order was delivered at 1:25 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nJustin M\nDasher\nClarisa\nPicked Up\n1:21 PM\n(Quoted: 1:20 PM)\nSeptember 25, 2023\nDropped Off\n1:25 PM\n(Quoted: 1:31 PM)\nSeptember 25, 2023\nPick Up Location\n401 N 1st St, Carrizo Springs, TX 78834, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nBig Red Bottle (20oz) (Soda TX)\n$3.79\n1\n×\nGatorade Fruit Punch (28 oz) (Sports Drinks TX)\n$4.79\nSubtotal\n$8.58\nSubtotal Tax\n$1.09\nCommission (16%)\n-$1.37\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$8.30\nAssociated Transactions (1)\nTransaction #8061018362 - Delivery\n$8.30',
 'Order: D6930CBB\nDelivered\nThe order was delivered at 1:05 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nPreston K\nDasher\nTelleia\nPicked Up\n12:54 PM\n(Quoted: 12:50 PM)\nSeptember 25, 2023\nDropped Off\n1:05 PM\n(Quoted: 1:32 PM)\nSeptember 25, 2023\nPick Up Location\n21024 W, 21024 State Hwy 71, Spicewood, TX 78669, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nChicken Fajita Taco (Lunch & Dinner Tacos TXB#62)\nSalsa:\nSalsa Roja\n$5.69\n1\n×\nMacaroni Cheese Medium (Sides TXB#62)\n$3.49\n1\n×\nChicken Tender Meal Kit (Meal Kits TXB#62)\n$9.19\n1\n×\nMash Potato & Gravy Large (Sides TXB#62)\n$5.99\nSubtotal\n$24.36\nSubtotal Tax\n$3.11\nCommission (16%)\n-$3.90\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$23.57\nAssociated Transactions (1)\nTransaction #8060830586 - Delivery\n$23.57',
 'Order: AB050AA9\nDelivered\nThe order was delivered at 12:36 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nHunter H\nDasher\nDavid\nPicked Up\n12:20 PM\n(Quoted: 12:15 PM)\nSeptember 25, 2023\nDropped Off\n12:36 PM\n(Quoted: 12:43 PM)\nSeptember 25, 2023\nPick Up Location\n3702 FM2147, Horseshoe Bay, TX 78657, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nArizona Green Tea Bottle (34oz) (Tea TX)\n$4.39\nSubtotal\n$4.39\nSubtotal Tax\n$0.00\nCommission (20%)\n-$0.88\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$3.51\nAssociated Transactions (1)\nTransaction #8060591450 - Delivery\n$3.51',
 'Order: 7A060210\nDelivered\nThe order was delivered at 12:20 PM on September 25, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nNeyssa P\nDasher\nAndrew\nPicked Up\n12:15 PM\n(Quoted: 12:17 PM)\nSeptember 25, 2023\nDropped Off\n12:20 PM\n(Quoted: 1:07 PM)\nSeptember 25, 2023\nPick Up Location\n14620 Ranch Rd 12, Wimberley, TX 78676, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nMash Potato & Gravy Medium (Sides TXB#57)\n$3.49\n2\n×\nHand-Breaded Chicken Roll Sandwich (Hot Sandwiches TXB#57)\nDipping Sauce:\nRanch Dipping Sauce\nOriginal or Spicy:\nSpicy\n$8.38\nSubtotal\n$11.87\nSubtotal Tax\n$1.51\nCommission (16%)\n-$1.90\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$11.48\nAssociated Transactions (1)\nTransaction #8060557422 - Delivery\n$11.48',
 'Order: 5F87EA62\nDelivered\nThe order was delivered at 10:14 AM on September 25, 2023. \nRate Dasher\nDelivery Details\nCustomer\nTrace M\nPicked Up\n10:08 AM\n(Quoted: 10:11 AM)\nSeptember 25, 2023\nPick Up Location\n1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA\nChannel\nDoorDash\nOrder Details\n1\n×\nChorizo Egg Cheese Quesadilla (Breakfast Tacos & Quesadillas)\nSalsa:\n1\n×\nDouble Bacon Egg Cheese Taco (Breakfast Tacos & Quesadillas)\nSalsa:\n1\n×\nC4 Frozen Bombscicle Can (16oz) (Energy Drinks TX)\n$4.79\n1\n×\nC4 Skittles Can (16oz) (Energy Drinks TX)\n$4.79\n1\n×\nC4 Starburst Cherry Can (16oz) (Energy Drinks TX)\n$4.79\n1\n×\nBreakfast Quesadilla (Breakfast Tacos & Quesadillas)\n$7.49\nSubtotal\n$35.24\nSubtotal Tax\n$5.81\nCommission (25%)\n-$8.81\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$32.24\nAssociated Transactions (1)\nTransaction #8059795042 - Delivery\n$32.24']

In [21]:
orders_content_cleaned = [order.replace('\n', '') for order in orders_content]

In [22]:
orders_content_cleaned

['Order: 7587448EDeliveredThe order was delivered at 11:15 PM on September 25, 2023. Rate DasherGet helpDelivery DetailsCustomermary rDasherMauricePicked Up11:00 PM(Quoted: 10:55 PM)September 25, 2023Dropped Off11:15 PM(Quoted: 11:22 PM)September 25, 2023Pick Up Location800 W Interstate 20, Big Spring, TX 79720, USAChannelDoorDashFulfillmentDoorDash DeliveryOrder Details2×TXB Water Bottle (1gal) (Water TX)$6.58Subtotal$6.58Subtotal Tax$0.00Commission (16%)-$1.05Total Customer Refund-$0.00Estimated Payout$5.53Associated Transactions (1)Transaction #8065464878 - Delivery$5.53',
 "Order: 51371086DeliveredThe order was delivered at 11:07 PM on September 25, 2023. Rate DasherGet helpDelivery DetailsCustomerChris WDasherLawrencePicked Up10:57 PM(Quoted: 10:37 PM)September 25, 2023Dropped Off11:07 PM(Quoted: 11:00 PM)September 25, 2023Pick Up Location102 W Main St, Calera, OK 74730, USAChannelDoorDashFulfillmentDoorDash DeliveryOrder Details1×Coke Cherry Bottle (20oz) (Soda OK)$3.791×Ben & Je

In [17]:
def get_order_id(order_text):
    # Extract the order_id
    order_id_start = order_text.find("Order:") + len("Order:")
    order_id_end = order_text.find("\n", order_id_start)
    order_id = order_text[order_id_start:order_id_end]
    return order_id

In [92]:
def orders_spreadsheet_creator(orders):
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
        
        # Loop through order_contents and create a sheet for each order
        for idx, order_text in enumerate(orders, start=1):
            
            # get order id
            order_id = get_order_id(order_text)
            
            # Convert the order text to a DataFrame
            df = pd.DataFrame(orders, columns=[f'{order_id}'])
            
            # Write the DataFrame to the Excel sheet with the order ID as the sheet name
            # TODO: store num getter func 
            df.to_excel(writer, sheet_name=f"store_num", index=False)
            
    # Close the Pandas Excel writer and save the file
    writer.save() #todo: AttributeError: 'XlsxWriter' object has no attribute 'save'


orders_spreadsheet_creator(orders_content)

AttributeError: 'XlsxWriter' object has no attribute 'save'

In [98]:
def create_excel_from_orders(order_strings):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
        for idx, order_string in enumerate(order_strings, start=1):
            # Create a DataFrame with the order string
            df = pd.DataFrame({'Order Content': [order_string]})
            df.to_clipboard()
            
            
            
            # Write the DataFrame to the Excel sheet with a sheet name based on the index
            # df.to_excel(writer, sheet_name=f"Order_{idx}", index=False)
    
    # Save the Excel file

# Example usage with a list of order strings
order_strings_with_delimiter = [
    'Order: DF8E5AB6 Delivered The order was delivered at 10:56 PM on September 24, 2023.  Rate Dasher Get help...',
    'Order: 1D4B1698 Delivered The order was delivered at 7:18 PM on September 24, 2023.  Rate Dasher Get help...'
]

create_excel_from_orders(order_strings_with_delimiter)


In [ ]:
import pyperclip
html = df.style.to_html()
pyperclip.copy(html)